In [1]:
import pandas as pd
import re
import os
from datetime import datetime
from dateutil import parser

In [2]:
log_name = '2024-09-23-1639.log'

# log_file_path = os.path.join('.', 'test', 'log-analysis-test', log_name)
log_file_path = os.path.join('.', 'wireshark_traffic', 'tshark', 'log', '2024-09-26-1444.log')
# output_log_path = os.path.join('.', 'test', 'log-analysis-test', 'cutoff_test.log')
# output_log_path = "Papers\\UserPattern\\wireshark_traffic\\tshark\\output_log\\cutoff-2024-09-26-1444.log"
output_log_path = os.path.join('.', 'wireshark_traffic', 'tshark', 'output_log', 'cutoff-2024-09-26-1444.log')

# process_port_path = "Papers\\UserPattern\\wireshark_order\\24-09-26--16-54-06.txt"
process_port_path = os.path.join('.', 'wireshark_order', '24-09-26--17-03-48.txt')

pattern = r'time="([^"]+)" level=[^ ]+ msg="([^"]+)"'
pattern_process = r'tcp\.port in \{([0-9, ]+)\}'

log_data = []
# process_port_tcp = []

In [3]:
with open(log_file_path, 'r') as file: 
    for line in file: 
        match = re.search(pattern, line) 
        if match: 
            log_entry = {
                'time': match.group(1), 
                'msg': match.group(2) 
            }
            log_data.append(log_entry)

In [4]:
df = pd.DataFrame(log_data, columns=['time', 'msg'])

In [5]:
with open(process_port_path, 'r') as file: # 提取filter order中的TCP端口号
    for line in file: 
        tcp_ports = re.search(pattern_process, line) 
        if tcp_ports: 
            process_port_tcp = [str(port.strip()) for port in tcp_ports.group(1).split(',')]

In [9]:
temp_msg = df['msg'].iloc[2736]
pattern_tcp = r'^\[TCP\]' # Starts with [TCP], ends with [DIRECT]
pattern_postfix = r'.*\[REJECT\]$'
pattern_conn = r'^inLoc'
if re.match(pattern_tcp, temp_msg): 
    if re.match(pattern_postfix, temp_msg): 
        print("True")
else: 
    print("False")

In [25]:
temp_msg = df['msg'].iloc[2736]
print(temp_msg)
patter_chrome = r'.*\(chrome\.exe\).*' 
port_pattern = r'.*\d{1,3}(?:\.\d{1,3}){3}:(\d+)'
if re.match(pattern_tcp, temp_msg): 
    if re.match(patter_chrome, temp_msg): 
        print("True") 
        match1 = re.search(port_pattern, temp_msg) 
        if match1: 
            port = match1.group(1) 
            print(port) 
        else: 
            print("Port capture error. ")
    else: 
        print("False")
else: 
    print("False")

[TCP] 198.18.0.1:62280(chrome.exe) --> pic1.zhimg.com:443 match DomainSuffix(zhimg.com) using 🧱直接连接[DIRECT]
True
62280


In [17]:
# 一次建立chrome和connection的列表，所有来自chrome的连接就以core为基准 
pattern_tcp = r'^\[TCP\]' 
pattern_chrome = r'.*\(chrome\.exe\).*' 
pattern_conn = r'^inLoc' 

list_chrome = [] 
list_connection = [] 
for i in range(df.shape[0]): 
    if re.match(pattern_tcp, df['msg'].iloc[i]): 
        if re.match(patter_chrome, df['msg'].iloc[i]): 
            list_chrome.append(df['msg'].iloc[i]) 
    elif re.match(pattern_conn, df['msg'].iloc[i]): 
        list_connection.append(df['msg'].iloc[i]) 

In [28]:
pattern_port = r'\[TCP\] \d{1,3}(?:\.\d{1,3}){3}:(\d+)' 
list_port = []
for i in range(len(list_chrome)): 
    match = re.search(pattern_port, list_chrome[i]) 
    if match: 
        port = match.group(1) 
        list_port.append(port) 
set_port = set(list_port) 

In [8]:
# 依次建立metadata和connection的列表，如果有的满足条件的metadata没打印出对应的connection信息怎么办
pattern_tcp = r'^\[TCP\]'
pattern_reject = r'.*\[REJECT\]$' 
pattern_conn = r'^inLoc' 
list_metadata = [] 
list_connection = [] 
for i in range(df.shape[0]): 
    if re.match(pattern_tcp, df['msg'].iloc[i]): 
        if not re.match(pattern_reject, df['msg'].iloc[i]): 
            list_metadata.append(df['msg'].iloc[i]) 
    elif re.match(pattern_conn, df['msg'].iloc[i]): 
        list_connection.append(df['msg'].iloc[i])

In [8]:
# Metadata信息拆解 ip, port. 进程不一定能匹配到所有的端口号，故交给捕捉脚本即可
meta_ip, meta_port = [], []
pattern_metainfo = r'(\d+\.\d+\.\d+\.\d+):(\d+)'
for entry in list_metadata: 
    match = re.search(pattern_metainfo, entry) # search()查找字符串中第一个匹配的部分
    if match: 
        meta_ip.append(match.group(1))
        meta_port.append(match.group(2))

In [9]:
# Connection信息拆解, inRemote: IP Port; inLoc: IP Port; outLoc: IP Port; outRemote: IP Port 
inRemoteIP, inRemotePort = [], []
inLocIP, inLocPort = [], []
outLocIP, outLocPort = [], []
outRemoteIP, outRemotePort= [], []
pattern_conninfo = r'(\d+\.\d+\.\d+\.\d+):(\d+)'

for entry in list_connection: 
    matches = re.findall(pattern_conninfo, entry) 
    # print()
    inRemoteIP.append(matches[1][0]), inRemotePort.append(matches[1][1])
    inLocIP.append(matches[0][0]), inLocPort.append(matches[0][1]) 
    outLocIP.append(matches[2][0]), outLocPort.append(matches[2][1])
    outRemoteIP.append(matches[3][0]), outRemotePort.append(matches[3][1])

In [22]:
df_connection = pd.DataFrame({
    'inRemoteIP': inRemoteIP,
    'inRemotePort': inRemotePort,
    'inLocIP': inLocIP,
    'inLocPort': inLocPort,
    'outLocIP': outLocIP,
    'outLocPort': outLocPort,
    'outRemoteIP': outRemoteIP,
    'outRemotePort': outRemotePort
})


In [23]:
df_conn_filter = df_connection[df_connection['inRemotePort'].isin(process_port_tcp)] # 需要将字符串类型转换为int

In [24]:
outboundPorts = set(df_conn_filter['outLocPort'])
tcp_filter = f"tcp.port in {{{', '.join(outboundPorts)}}}"

In [9]:
# 指定截断时间
cutoff_time_pre = parser.isoparse("2024-09-23T16:43:39.5558452+08:00")
# cutoff_time_post = parser.isoparse("2024-09-23T16:44:05.1522054+08:00")

# 打开日志文件
with open(log_file_path, 'r') as file, open(output_log_path, 'w') as output:
    for line in file:
        # 提取时间戳
        time_str = line.split(' ')[0].split('=')[1].strip('"')
        log_time = parser.isoparse(time_str)
        
        # 检查时间是否在指定时间之后
        if log_time >= cutoff_time_pre: 
            output.write(line)
